# Introduction

Dans ce projet, nous avons implémenté un modèle de deep learning pour pouvoir détecter les machines malicieuses sur un réseau informatique. Nous avons utilisé le 'botnet dataset' de l'Université du Nouveau-Brunswick (UNB). Un botnet est un réseau informatique infecté par des personnes externes qui utilise la puissance de plusieurs ordinateurs pour effectuer des actions malveillantes. Pour commencer, j'ai lu des papiers de recherche sur la detéction de botnet à l'aide de l'intelligence artificielle. Par la suite, j'ai du faire le nettoyage des données ainsi que la selection des features à garder ainsi que la génération de features. Pour finir, j'ai utilisé la librairies keras pour faire mon modèle.

# Librairies

Lors de ce projet, j'ai pu toucher à plusieurs librairies connus dans le monde de science de donnée. Tout au long du projet, j'ai travaillé avec la librairie pandas pour manipuler les dataframes et créer les features de mon modèle.

In [ ]:
import keras
import pandas as pd
from keras import regularizers
from sklearn.preprocessing import StandardScaler

# Data cleansing

In [ ]:
def rename_dataframe_column(dataframe_rename_columns):
    for column in dataframe_rename_columns.columns:
        new_column_name = column.replace(" ", "_").replace('/', '_').lower()
        # new_column_name = column.replace("/", "_")
        dataframe_rename_columns.rename(index=str, columns={column: new_column_name}, inplace=True)


def clean_dataset(dataset_to_clean):
    malicious_ips = ['192.168.2.112', '198.164.30.2', '192.168.2.113', '192.168.2.112', '147.32.84.180',
                     '147.32.84.140', '10.0.2.15',
                     '172.16.253.130', '172.16.253.240', '192.168.3.35', '172.29.0.116', '192.168.248.165',
                     '131.202.243.84',
                     '192.168.2.110', '192.168.1.103', '192.168.2.109', '147.32.84.170', '147.32.84.130',
                     '192.168.106.141',
                     '172.16.253.131', '74.78.117.238', '192.168.3.25', '172.29.0.109', '10.37.130.4', '192.168.5.122',
                     '192.168.4.118',
                     '192.168.4.120', '192.168.2.105', '147.32.84.150', '147.32.84.160', '192.168.106.131',
                     '172.16.253.129',
                     '158.65.110.24', '192.168.3.65', '172.16.253.132']

    
    # iterate through dataframe and set value
    for row in dataset_to_clean.itertuples():
        if ((dataset_to_clean.at[row.Index, 'src_ip'] in malicious_ips) or (
                dataset_to_clean.at[row.Index, 'dst_ip'] in malicious_ips)):
            dataset_to_clean.at[row.Index, 'label'] = 1
        else:
            dataset_to_clean.at[row.Index, 'label'] = 0

    # Drop row with infinity string value
    dataset_to_clean.drop(dataset_to_clean.loc[dataset_to_clean['flow_byts_s'] == "Infinity"].index, inplace=True)
    dataset_to_clean.drop(dataset_to_clean.loc[dataset_to_clean['flow_pkts_s'] == "Infinity"].index, inplace=True)

    # Pour drop colonne completement
    to_drop = ['flow_id', 'src_ip', 'src_port', 'dst_ip', 'dst_port', 'timestamp']

    to_drop_feature_selection = ['flow_id', 'src_ip', 'src_port', 'dst_ip', 'dst_port', 'timestamp']

    # inplace = true, fait en sorte que c est drop directement dans l objet
    dataset_to_clean.drop(to_drop_feature_selection, axis=1, inplace=True)

    # dataset.dropna(subset = ['Src IP','Src Port','Dst IP','Dst Port'])
    dataset_to_clean.dropna(how="any", axis=0, inplace=True)

    # cast flow pkts and
    dataset_to_clean[["flow_byts_s", "flow_pkts_s"]] = \
        dataset_to_clean[["flow_byts_s", "flow_pkts_s"]].apply(pd.to_numeric)

# méthode utilisé avec le nouveau dataset
def clean_dataset2(dataset_to_clean):
    to_drop_feature_selection = ['StartTime', 'Proto', 'SrcAddr', 'Sport', 'DstAddr', 'Dport','unix_time']
    dataset_to_clean.drop(to_drop_feature_selection, axis=1, inplace=True)

# Feature selection

Durant cette partie du projet, j'ai voulu essayer d'enlever quelques features qui pourraient être inutile. J'ai donc fait une matrice de corrélation entre les features pour pouvoir éliminer des features qui sont fortement corrélés entre eux. Après ce processus, j'ai conclu que cétait mieux de garder tous les features. Pour pouvoir le code pour la matrice de corrélation, référez-vous au notebook Feature Correlations.

# Feature Generation

Durant cette phase du projet, on m'a assigné la tâche de générer des features pour un autre projet. Je n'ai donc pas eu la chance de faire les features pour ce projet. Par contre, on m'a donné le fichier csv à utiliser qui contient des nouveaux features. Voici la liste de features qui a était ajouté.

### Connection-based features : 

(Using a rolling window for the previous n netflows when a given source or destination address appears in the traffic)

- For any of the flow records that SRCADDRESS has appeared within the last n flow records, average the bytes (A_TotBytes_S)
- For any of the flow records that SRCADDRESS has appeared within the last n flow records, average the packets (A_TotPkts_S)
- Number of apperance of SRCADDRESS in the last n/10 netflows (Nbr_App_S)
- For any of the flow records that SRCADDRESS has appeared within the last n flow records, count the distinct source ports (Dct_Sport_S)
- For any of the flow records that SRCADDRESS has appeared within the last n flow records, count the distinct destination ports (Dct_Dport_S)
- For any of the flow records that SRCADDRESS has appeared within the last n flow records, count the distinct source ips (Dct_SrcAddr_S)

- For any of the flow records that DSTADDRESS has appeared within the last n flow records, average the bytes (A_TotBytes_D)
- For any of the flow records that DSTADDRESS has appeared within the last n flow records, average the packets (A_TotPkts_D)
- Number of apperance of DSTADDRESS in the last n/10 netflows (Nbr_App_D)
- For any of the flow records that DSTADDRESS has appeared within the last n flow records, count the distinct source ports (Dct_Sport_D)
- For any of the flow records that DSTADDRESS has appeared within the last n flow records, count the distinct destination ports (Dct_Dport_D)
- For any of the flow records that DSTADDRESS has appeared within the last n flow records, count the distinct destination ips (Dct_DstAddr_D)

###  Time-based features : 

(Using a rolling window for the n previous minutes when a given source or destination address appears in the traffic). 

- For any of the flow records that SRCADDRESS has appeared within the last n minutes, average the bytes (A_TotBytes_S)
- For any of the flow records that SRCADDRESS has appeared within the last n minutes, average the packets (A_TotPkts_S)
- For any of the flow records that SRCADDRESS has appeared within the last n minutes, count the distinct source ports (Dct_Sport_S)
- For any of the flow records that SRCADDRESS has appeared within the last n minutes, count the distinct source ips (Dct_SrcAddr_S)
- Number of apperance of SRCADDRESS  within the last n/10 minutes (Nbr_App_S)

- For any of the flow records that DSTADDRESS has appeared within the last n minutes, average the bytes (A_TotBytes_D)
- For any of the flow records that DSTADDRESS has appeared within the last n minutes, average the packets (A_TotPkts_D)
- For any of the flow records that DSTADDRESS has appeared within the last n minutes, count the distinct source ports (Dct_Sport_D)
- For any of the flow records that DSTADDRESS has appeared within the last n minutes, count the distinct destination ips (Dct_DstAddr_D)
- Number of apperance of DSTADDRESS  within the last n/10 minutes (Nbr_App_D)


# Modèle

In [ ]:
# dataset_training = pd.read_csv('./ISCX_Botnet-Training.pcap_Flow_ubuntu.csv', encoding='utf-8', low_memory=False)
# dataset_testing = pd.read_csv('./ISCX_Botnet-Testing.pcap_Flow.csv', encoding='utf-8', low_memory=False)

dataset_training = pd.read_csv('unb-Training.csv', encoding='utf-8', low_memory=False)
dataset_testing = pd.read_csv('unb-testing.csv', encoding='utf-8', low_memory=False)

clean_dataset2(dataset_testing)
clean_dataset2(dataset_training)

# Split data
x_data_training = dataset_training.drop('Label', axis=1)
labels_training = dataset_training['Label']
x_data_testing = dataset_testing.drop('Label', axis=1)
labels_testing = dataset_testing['Label']

# Normalize data
std = StandardScaler()
x_data_training = std.fit_transform(x_data_training)
x_data_testing = std.fit_transform(x_data_testing)

# Create Model
model = keras.Sequential()
# Input Layer Layer and first hidden layer
model.add(keras.layers.Dense(9, activation='relu', input_shape=(18,), kernel_regularizer= regularizers.l2(0.01)))
model.add(keras.layers.Dropout(0.4))
# Output Layer
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.001), metrics=["accuracy"])

es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# Résultat

In [13]:
model.fit(x_data_training, labels_training, epochs=2000, batch_size=500, shuffle= True,
                             validation_data=(x_data_testing, labels_testing), verbose=2)

predictions = model.predict(x_data_testing, batch_size=100, verbose=0)

print(predictions)

Train on 353396 samples, validate on 300862 samples
Epoch 1/2000
 - 6s - loss: 0.1741 - acc: 0.9439 - val_loss: 5.5110 - val_acc: 0.3438
Epoch 2/2000
 - 5s - loss: 0.1741 - acc: 0.9440 - val_loss: 5.3409 - val_acc: 0.3575
Epoch 3/2000
 - 5s - loss: 0.1742 - acc: 0.9437 - val_loss: 5.4799 - val_acc: 0.3848
Epoch 4/2000
 - 5s - loss: 0.1752 - acc: 0.9437 - val_loss: 5.3682 - val_acc: 0.3706
Epoch 5/2000
 - 5s - loss: 0.1746 - acc: 0.9441 - val_loss: 5.4473 - val_acc: 0.3595
Epoch 6/2000
 - 5s - loss: 0.1742 - acc: 0.9436 - val_loss: 5.7118 - val_acc: 0.3727
Epoch 7/2000
 - 5s - loss: 0.1752 - acc: 0.9439 - val_loss: 5.5719 - val_acc: 0.3654
Epoch 8/2000
 - 5s - loss: 0.1747 - acc: 0.9437 - val_loss: 5.4923 - val_acc: 0.3624
Epoch 9/2000
 - 5s - loss: 0.1749 - acc: 0.9437 - val_loss: 5.0992 - val_acc: 0.3796
Epoch 10/2000
 - 5s - loss: 0.1760 - acc: 0.9430 - val_loss: 5.1690 - val_acc: 0.3685
Epoch 11/2000
 - 5s - loss: 0.1750 - acc: 0.9431 - val_loss: 5.4152 - val_acc: 0.3562
Epoch 12/20

Epoch 96/2000
 - 10s - loss: 0.1702 - acc: 0.9465 - val_loss: 5.1782 - val_acc: 0.3561
Epoch 97/2000
 - 17s - loss: 0.1709 - acc: 0.9466 - val_loss: 5.0618 - val_acc: 0.3832
Epoch 98/2000
 - 10s - loss: 0.1695 - acc: 0.9466 - val_loss: 5.0472 - val_acc: 0.3888
Epoch 99/2000
 - 6s - loss: 0.1695 - acc: 0.9464 - val_loss: 5.5065 - val_acc: 0.3858
Epoch 100/2000
 - 7s - loss: 0.1698 - acc: 0.9467 - val_loss: 4.9456 - val_acc: 0.3905
Epoch 101/2000
 - 6s - loss: 0.1710 - acc: 0.9461 - val_loss: 5.3077 - val_acc: 0.3885
Epoch 102/2000
 - 6s - loss: 0.1704 - acc: 0.9461 - val_loss: 5.1411 - val_acc: 0.3577
Epoch 103/2000
 - 9s - loss: 0.1695 - acc: 0.9468 - val_loss: 5.7925 - val_acc: 0.3776
Epoch 104/2000
 - 7s - loss: 0.1702 - acc: 0.9467 - val_loss: 5.1476 - val_acc: 0.3915
Epoch 105/2000
 - 5s - loss: 0.1695 - acc: 0.9465 - val_loss: 5.2500 - val_acc: 0.3829
Epoch 106/2000
 - 5s - loss: 0.1714 - acc: 0.9463 - val_loss: 5.5825 - val_acc: 0.3694
Epoch 107/2000
 - 5s - loss: 0.1707 - acc: 0

 - 5s - loss: 0.1694 - acc: 0.9463 - val_loss: 5.2538 - val_acc: 0.3709
Epoch 191/2000
 - 5s - loss: 0.1690 - acc: 0.9466 - val_loss: 5.4121 - val_acc: 0.3783
Epoch 192/2000
 - 5s - loss: 0.1688 - acc: 0.9464 - val_loss: 5.2037 - val_acc: 0.3868
Epoch 193/2000
 - 5s - loss: 0.1686 - acc: 0.9463 - val_loss: 5.5349 - val_acc: 0.3602
Epoch 194/2000
 - 5s - loss: 0.1684 - acc: 0.9466 - val_loss: 5.3719 - val_acc: 0.3874
Epoch 195/2000
 - 5s - loss: 0.1688 - acc: 0.9462 - val_loss: 5.6903 - val_acc: 0.3793
Epoch 196/2000
 - 5s - loss: 0.1689 - acc: 0.9462 - val_loss: 5.7244 - val_acc: 0.3742
Epoch 197/2000
 - 5s - loss: 0.1684 - acc: 0.9467 - val_loss: 5.1586 - val_acc: 0.3708
Epoch 198/2000
 - 5s - loss: 0.1690 - acc: 0.9465 - val_loss: 5.7376 - val_acc: 0.3774
Epoch 199/2000
 - 5s - loss: 0.1674 - acc: 0.9469 - val_loss: 5.6236 - val_acc: 0.3689
Epoch 200/2000
 - 5s - loss: 0.1678 - acc: 0.9465 - val_loss: 5.5780 - val_acc: 0.3753
Epoch 201/2000
 - 5s - loss: 0.1685 - acc: 0.9469 - val_lo

Epoch 285/2000
 - 7s - loss: 0.1693 - acc: 0.9459 - val_loss: 5.5930 - val_acc: 0.3822
Epoch 286/2000
 - 7s - loss: 0.1689 - acc: 0.9463 - val_loss: 5.3424 - val_acc: 0.3733
Epoch 287/2000
 - 6s - loss: 0.1668 - acc: 0.9465 - val_loss: 5.2965 - val_acc: 0.3733
Epoch 288/2000
 - 94072s - loss: 0.1683 - acc: 0.9463 - val_loss: 5.4250 - val_acc: 0.3878
Epoch 289/2000
 - 8s - loss: 0.1669 - acc: 0.9465 - val_loss: 5.4599 - val_acc: 0.3777
Epoch 290/2000
 - 3s - loss: 0.1685 - acc: 0.9462 - val_loss: 5.6368 - val_acc: 0.3859
Epoch 291/2000
 - 3s - loss: 0.1688 - acc: 0.9461 - val_loss: 5.3079 - val_acc: 0.3900
Epoch 292/2000
 - 5s - loss: 0.1669 - acc: 0.9466 - val_loss: 5.7447 - val_acc: 0.3812
Epoch 293/2000
 - 3s - loss: 0.1676 - acc: 0.9466 - val_loss: 5.9339 - val_acc: 0.3849
Epoch 294/2000
 - 3s - loss: 0.1684 - acc: 0.9460 - val_loss: 5.4497 - val_acc: 0.3835
Epoch 295/2000
 - 4s - loss: 0.1671 - acc: 0.9464 - val_loss: 5.3828 - val_acc: 0.3839
Epoch 296/2000
 - 4s - loss: 0.1669 - a

 - 2s - loss: 0.1670 - acc: 0.9464 - val_loss: 5.4132 - val_acc: 0.3814
Epoch 380/2000
 - 2s - loss: 0.1667 - acc: 0.9464 - val_loss: 5.3858 - val_acc: 0.3871
Epoch 381/2000
 - 2s - loss: 0.1662 - acc: 0.9465 - val_loss: 5.3207 - val_acc: 0.3759
Epoch 382/2000
 - 2s - loss: 0.1677 - acc: 0.9464 - val_loss: 5.5124 - val_acc: 0.3882
Epoch 383/2000
 - 3s - loss: 0.1674 - acc: 0.9461 - val_loss: 5.3068 - val_acc: 0.3840
Epoch 384/2000
 - 2s - loss: 0.1674 - acc: 0.9457 - val_loss: 5.3956 - val_acc: 0.3777
Epoch 385/2000
 - 2s - loss: 0.1670 - acc: 0.9466 - val_loss: 5.4737 - val_acc: 0.3915
Epoch 386/2000
 - 2s - loss: 0.1656 - acc: 0.9468 - val_loss: 5.4487 - val_acc: 0.3770
Epoch 387/2000
 - 2s - loss: 0.1664 - acc: 0.9462 - val_loss: 5.7531 - val_acc: 0.3816
Epoch 388/2000
 - 4s - loss: 0.1667 - acc: 0.9465 - val_loss: 5.3776 - val_acc: 0.3673
Epoch 389/2000
 - 3s - loss: 0.1666 - acc: 0.9470 - val_loss: 5.5278 - val_acc: 0.3841
Epoch 390/2000
 - 3s - loss: 0.1659 - acc: 0.9463 - val_lo

...

Epoch 1601/2000
 - 2s - loss: 0.1660 - acc: 0.9463 - val_loss: 5.4391 - val_acc: 0.3855
Epoch 1602/2000
 - 2s - loss: 0.1657 - acc: 0.9467 - val_loss: 5.3295 - val_acc: 0.3897
Epoch 1603/2000
 - 2s - loss: 0.1667 - acc: 0.9465 - val_loss: 5.9817 - val_acc: 0.3767
Epoch 1604/2000
 - 2s - loss: 0.1661 - acc: 0.9457 - val_loss: 5.5557 - val_acc: 0.3751
Epoch 1605/2000
 - 2s - loss: 0.1665 - acc: 0.9453 - val_loss: 6.0850 - val_acc: 0.3705
Epoch 1606/2000
 - 2s - loss: 0.1656 - acc: 0.9463 - val_loss: 5.4805 - val_acc: 0.3568
Epoch 1607/2000
 - 2s - loss: 0.1649 - acc: 0.9461 - val_loss: 5.7868 - val_acc: 0.3751
Epoch 1608/2000
 - 2s - loss: 0.1665 - acc: 0.9458 - val_loss: 5.3632 - val_acc: 0.3822
Epoch 1609/2000
 - 2s - loss: 0.1652 - acc: 0.9463 - val_loss: 5.5056 - val_acc: 0.3759
Epoch 1610/2000
 - 2s - loss: 0.1659 - acc: 0.9461 - val_loss: 5.7433 - val_acc: 0.3848
Epoch 1611/2000
 - 2s - loss: 0.1668 - acc: 0.9460 - val_loss: 6.1620 - val_acc: 0.3742
Epoch 1612/2000
 - 2s - loss: 0.

 - 2s - loss: 0.1657 - acc: 0.9463 - val_loss: 5.9210 - val_acc: 0.3802
Epoch 1695/2000
 - 2s - loss: 0.1666 - acc: 0.9464 - val_loss: 6.3348 - val_acc: 0.3837
Epoch 1696/2000
 - 2s - loss: 0.1658 - acc: 0.9464 - val_loss: 5.2452 - val_acc: 0.3766
Epoch 1697/2000
 - 2s - loss: 0.1651 - acc: 0.9468 - val_loss: 5.4566 - val_acc: 0.3748
Epoch 1698/2000
 - 2s - loss: 0.1660 - acc: 0.9464 - val_loss: 5.5859 - val_acc: 0.3613
Epoch 1699/2000
 - 2s - loss: 0.1665 - acc: 0.9455 - val_loss: 5.9530 - val_acc: 0.3727
Epoch 1700/2000
 - 2s - loss: 0.1667 - acc: 0.9459 - val_loss: 6.1793 - val_acc: 0.3717
Epoch 1701/2000
 - 2s - loss: 0.1650 - acc: 0.9467 - val_loss: 6.0862 - val_acc: 0.3605
Epoch 1702/2000
 - 2s - loss: 0.1654 - acc: 0.9468 - val_loss: 5.4752 - val_acc: 0.3822
Epoch 1703/2000
 - 2s - loss: 0.1652 - acc: 0.9463 - val_loss: 5.3076 - val_acc: 0.3833
Epoch 1704/2000
 - 2s - loss: 0.1654 - acc: 0.9471 - val_loss: 5.4148 - val_acc: 0.3677
Epoch 1705/2000
 - 2s - loss: 0.1661 - acc: 0.94

Epoch 1788/2000
 - 2s - loss: 0.1650 - acc: 0.9463 - val_loss: 5.9719 - val_acc: 0.3844
Epoch 1789/2000
 - 2s - loss: 0.1670 - acc: 0.9461 - val_loss: 6.0463 - val_acc: 0.3781
Epoch 1790/2000
 - 2s - loss: 0.1656 - acc: 0.9465 - val_loss: 5.3416 - val_acc: 0.3699
Epoch 1791/2000
 - 2s - loss: 0.1650 - acc: 0.9463 - val_loss: 5.4445 - val_acc: 0.3848
Epoch 1792/2000
 - 2s - loss: 0.1653 - acc: 0.9463 - val_loss: 5.7285 - val_acc: 0.3811
Epoch 1793/2000
 - 2s - loss: 0.1644 - acc: 0.9464 - val_loss: 5.2104 - val_acc: 0.3754
Epoch 1794/2000
 - 2s - loss: 0.1671 - acc: 0.9462 - val_loss: 6.0950 - val_acc: 0.3667
Epoch 1795/2000
 - 2s - loss: 0.1650 - acc: 0.9468 - val_loss: 5.5166 - val_acc: 0.3862
Epoch 1796/2000
 - 2s - loss: 0.1658 - acc: 0.9463 - val_loss: 5.5283 - val_acc: 0.3859
Epoch 1797/2000
 - 2s - loss: 0.1652 - acc: 0.9467 - val_loss: 5.7423 - val_acc: 0.3828
Epoch 1798/2000
 - 2s - loss: 0.1669 - acc: 0.9458 - val_loss: 5.4262 - val_acc: 0.3616
Epoch 1799/2000
 - 2s - loss: 0.

 - 2s - loss: 0.1670 - acc: 0.9457 - val_loss: 5.2163 - val_acc: 0.3916
Epoch 1882/2000
 - 2s - loss: 0.1668 - acc: 0.9463 - val_loss: 5.9922 - val_acc: 0.3753
Epoch 1883/2000
 - 2s - loss: 0.1656 - acc: 0.9461 - val_loss: 5.7761 - val_acc: 0.3655
Epoch 1884/2000
 - 2s - loss: 0.1659 - acc: 0.9465 - val_loss: 5.7669 - val_acc: 0.3587
Epoch 1885/2000
 - 2s - loss: 0.1668 - acc: 0.9461 - val_loss: 6.2550 - val_acc: 0.3656
Epoch 1886/2000
 - 2s - loss: 0.1657 - acc: 0.9462 - val_loss: 6.0404 - val_acc: 0.3805
Epoch 1887/2000
 - 2s - loss: 0.1654 - acc: 0.9463 - val_loss: 5.8116 - val_acc: 0.3862
Epoch 1888/2000
 - 2s - loss: 0.1672 - acc: 0.9461 - val_loss: 5.7209 - val_acc: 0.3814
Epoch 1889/2000
 - 2s - loss: 0.1656 - acc: 0.9466 - val_loss: 5.2356 - val_acc: 0.3895
Epoch 1890/2000
 - 2s - loss: 0.1657 - acc: 0.9463 - val_loss: 5.3942 - val_acc: 0.3814
Epoch 1891/2000
 - 2s - loss: 0.1654 - acc: 0.9467 - val_loss: 5.7940 - val_acc: 0.3854
Epoch 1892/2000
 - 2s - loss: 0.1667 - acc: 0.94

Epoch 1975/2000
 - 2s - loss: 0.1665 - acc: 0.9461 - val_loss: 5.9322 - val_acc: 0.3733
Epoch 1976/2000
 - 2s - loss: 0.1638 - acc: 0.9471 - val_loss: 5.5297 - val_acc: 0.3728
Epoch 1977/2000
 - 2s - loss: 0.1652 - acc: 0.9467 - val_loss: 5.4346 - val_acc: 0.3841
Epoch 1978/2000
 - 2s - loss: 0.1660 - acc: 0.9460 - val_loss: 5.9282 - val_acc: 0.3700
Epoch 1979/2000
 - 2s - loss: 0.1648 - acc: 0.9467 - val_loss: 4.9773 - val_acc: 0.3871
Epoch 1980/2000
 - 2s - loss: 0.1661 - acc: 0.9466 - val_loss: 5.7264 - val_acc: 0.3747
Epoch 1981/2000
 - 2s - loss: 0.1666 - acc: 0.9460 - val_loss: 5.4743 - val_acc: 0.3797
Epoch 1982/2000
 - 2s - loss: 0.1649 - acc: 0.9468 - val_loss: 5.4614 - val_acc: 0.3601
Epoch 1983/2000
 - 2s - loss: 0.1668 - acc: 0.9462 - val_loss: 5.8508 - val_acc: 0.3616
Epoch 1984/2000
 - 2s - loss: 0.1670 - acc: 0.9462 - val_loss: 5.7214 - val_acc: 0.3609
Epoch 1985/2000
 - 2s - loss: 0.1664 - acc: 0.9464 - val_loss: 5.1845 - val_acc: 0.3836
Epoch 1986/2000
 - 2s - loss: 0.

Comme on peut le voir, les résultats sont assez décevant. En training, le modèle est capable d'atteindre une accuracy de 99%, mais en testing celui-ci reste entre 40% et 50%.

# Dans le futur